# Gold Trading Prediction with LSTM

What is LSTM?

LSTM = Long Short-Term Memory
LSTM networks are specially designed to remember and process sequences of data over a long period. They are better than traditional RNNs. 
*	Can preserve information for long durations. 
*	Unique structure comprising three gates: the input, forget, and output gates.
*	These gates collaboratively manage the flow of information, deciding what to retain and what to discard, by mitigating the issue of vanishing gradients, a common problem in standard RNNs.

LSTM networks adeptly capture temporal dependencies, making them ideal for financial time series analysis.


In [1]:
# import required libaraies
import tensorflow as tf
import keras
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# model packages
from sklearn.preprocessing import MinMaxScaler

In [2]:
# download the required data from yfinance
gold_one_day = yf.download('GC=F', start="2024-06-04", end="2024-06-05", interval="1m")
gold_one_year = yf.download('GC=F', period='1y')
gold_ten_year = yf.download('GC=F', period='10y')
print("Gold data are successfully downloaded.")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Gold data are successfully downloaded.


In [3]:
# checkout gold data for one day
gold_one_day.head(5)

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-06-04 00:00:00-04:00,2366.000000,2366.399902,2365.399902,2365.899902,2365.899902,0
2024-06-04 00:01:00-04:00,2365.899902,2366.300049,2365.600098,2366.100098,2366.100098,199
2024-06-04 00:02:00-04:00,2366.000000,2366.699951,2365.800049,2366.699951,2366.699951,132
2024-06-04 00:03:00-04:00,2366.800049,2366.899902,2366.300049,2366.899902,2366.899902,56
2024-06-04 00:04:00-04:00,2366.800049,2367.100098,2366.800049,2367.000000,2367.000000,86


In [4]:
# checkout gold data for one year
gold_one_year.head(5)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-06-12,1959.699951,1963.500000,1951.400024,1955.300049,1955.300049,212
2023-06-13,1960.599976,1967.500000,1940.300049,1944.599976,1944.599976,343
2023-06-14,1946.199951,1958.199951,1940.300049,1955.300049,1955.300049,202
2023-06-15,1941.699951,1958.800049,1926.000000,1957.800049,1957.800049,329
2023-06-16,1961.400024,1962.900024,1953.500000,1958.400024,1958.400024,119


In [5]:
# checkout gold data for one year
gold_ten_year.head(5)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-06-12,1260.000000,1274.500000,1260.000000,1273.599976,1273.599976,129
2014-06-13,1272.500000,1274.699951,1271.300049,1273.699951,1273.699951,145
2014-06-16,1279.800049,1282.099976,1270.800049,1274.900024,1274.900024,334
2014-06-17,1272.800049,1272.800049,1260.900024,1271.699951,1271.699951,819
2014-06-18,1269.900024,1272.699951,1267.400024,1272.400024,1272.400024,143


In [6]:
# check out if there is missing value 
print("Gold data one day")
print(gold_one_day.isna().sum())
print("------------------")
print("Gold data one year")
print(gold_one_year.isna().sum())
print("------------------")
print("Gold data ten years")
print(gold_ten_year.isna().sum())

Gold data one day
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64
------------------
Gold data one year
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64
------------------
Gold data ten years
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64


In [7]:
scaler = MinMaxScaler(feature_range=(0,1))
gold_one_day_scaled = scaler.fit_transform(gold_one_day['Close'].values.reshape(-1,1))

In [8]:
gold_one_day_scaled

array([[0.84444173],
       [0.85000271],
       [0.86666531],
       ...,
       [0.59166802],
       [0.59166802],
       [0.59444173]])

In [9]:
gold_one_day_scaled.shape

(1364, 1)

In [10]:
# initialize empty lists
X = []
y = []
# loop to create a 60-1min-sequence
for i in range(60, len(gold_one_day_scaled)):
    X.append(gold_one_day_scaled[i-60:i, 0])
    y.append(gold_one_day_scaled[i, 0])

In [11]:
# Train-Test-Split
train_size = int(len(X) * 0.8)
test_size = len(X) - train_size

X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [12]:
# reshaping to train in LSTM model ([samples, time steps, features]) 
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [13]:
# LSTM model
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, AdditiveAttention, Permute, Reshape, Multiply

model = Sequential()

# Adding LSTM layers with return_sequences=True
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50, return_sequences=True))

In [14]:
# Adding self-attention mechanism
# The attention mechanism
attention = AdditiveAttention(name='attention_weight')
# Permute and reshape for compatibility
model.add(Permute((2, 1))) 
model.add(Reshape((-1, X_train.shape[1])))
# compute attention result 
attention_result = attention([model.output, model.output])
# perform element-wise multiplication of the model's output and the attention result
multiply_layer = Multiply()([model.output, attention_result])
# Adding a Flatten layer before the final Dense layer
model.add(tf.keras.layers.Flatten())
# Final Dense layer
model.add(Dense(1))

In [15]:
from keras.layers import BatchNormalization

# Adding Dropout and Batch Normalization
"""Dropout helps in preventing overfitting by randomly setting a fraction of the input 
units to 0 at each update during training, and Batch Normalization stabilizes the learning 
process."""
model.add(Dropout(0.2))
model.add(BatchNormalization())

In [16]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [17]:
# review the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 60, 50)            10400     
                                                                 
 lstm_1 (LSTM)               (None, 60, 50)            20200     
                                                                 
 permute (Permute)           (None, 50, 60)            0         
                                                                 
 reshape (Reshape)           (None, 50, 60)            0         
                                                                 
 flatten (Flatten)           (None, 3000)              0         
                                                                 
 dense (Dense)               (None, 1)                 3001      
                                                                 
 dropout (Dropout)           (None, 1)                 0

In [18]:
# Assuming X_train and y_train are already defined and preprocessed
history = model.fit(X_train, y_train, epochs=100, batch_size=25, validation_split=0.2)

Epoch 1/100
34/34 [==============================] - 4s 43ms/step - loss: 0.4403 - val_loss: 0.0821
Epoch 2/100
34/34 [==============================] - 1s 28ms/step - loss: 0.1908 - val_loss: 0.0632
Epoch 3/100
34/34 [==============================] - 1s 28ms/step - loss: 0.1669 - val_loss: 0.0528
Epoch 4/100
34/34 [==============================] - 1s 26ms/step - loss: 0.1450 - val_loss: 0.0364
Epoch 5/100
34/34 [==============================] - 1s 27ms/step - loss: 0.1244 - val_loss: 0.0263
Epoch 6/100
34/34 [==============================] - 1s 32ms/step - loss: 0.1101 - val_loss: 0.0208
Epoch 7/100
34/34 [==============================] - 1s 39ms/step - loss: 0.0898 - val_loss: 0.0137
Epoch 8/100
34/34 [==============================] - 1s 34ms/step - loss: 0.0762 - val_loss: 0.0093
Epoch 9/100
34/34 [==============================] - 1s 33ms/step - loss: 0.0654 - val_loss: 0.0078
Epoch 10/100
34/34 [==============================] - 1s 34ms/step - loss: 0.0580 - val_loss: 0.0033

In [19]:
# Convert X_test and y_test to Numpy arrays if they are not already
X_test = np.array(X_test)
y_test = np.array(y_test)

# Ensure X_test is reshaped similarly to how X_train was reshaped
# This depends on how you preprocessed the training data
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Now evaluate the model on the test data
test_loss = model.evaluate(X_test, y_test)
print("Test Loss: ", test_loss)

9/9 [==============================] - 0s 21ms/step - loss: 0.0026
Test Loss:  0.0025574816390872


In [20]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Making predictions
y_pred = model.predict(X_test)

# Calculating MAE and RMSE
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print("Mean Absolute Error: ", mae)
print("Root Mean Square Error: ", rmse)


9/9 [==============================] - 4s 15ms/step
Mean Absolute Error:  0.04536230282968522
Root Mean Square Error:  0.05057154919235595


In [21]:
# Fetching the latest 60 days of AAPL stock data
test_data = yf.download('GC=F', interval='1m', period='1D')

[*********************100%%**********************]  1 of 1 completed


In [22]:
test_data.head(5)

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-06-12 00:00:00-04:00,2330.500000,2330.600098,2330.199951,2330.300049,2330.300049,0
2024-06-12 00:01:00-04:00,2330.199951,2330.600098,2330.199951,2330.600098,2330.600098,18
2024-06-12 00:02:00-04:00,2330.500000,2330.500000,2330.399902,2330.399902,2330.399902,4
2024-06-12 00:03:00-04:00,2330.300049,2330.500000,2330.300049,2330.500000,2330.500000,7
2024-06-12 00:04:00-04:00,2330.500000,2330.899902,2330.500000,2330.800049,2330.800049,13


In [23]:
test_data.tail(5)

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-06-12 00:44:00-04:00,2329.699951,2330.000000,2329.699951,2330.000000,2330.000000,15
2024-06-12 00:45:00-04:00,2330.000000,2330.199951,2330.000000,2330.199951,2330.199951,19
2024-06-12 00:46:00-04:00,2330.199951,2330.199951,2330.100098,2330.199951,2330.199951,11
2024-06-12 00:47:00-04:00,2330.100098,2330.399902,2330.100098,2330.199951,2330.199951,20
2024-06-12 00:48:00-04:00,2330.199951,2330.300049,2330.199951,2330.199951,2330.199951,0


In [24]:
# Selecting the 'Close' price and converting to numpy array
closing_prices = test_data['Close'].values

# Scaling the data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(closing_prices.reshape(-1,1))

# Since we need the last 60 days to predict the next day, we reshape the data accordingly
X_latest = np.array([scaled_data[-60:].reshape(60)])

# Reshaping the data for the model (adding batch dimension)
X_latest = np.reshape(X_latest, (X_latest.shape[0], X_latest.shape[1], 1))

# Making predictions for the next 4 candles
predicted_stock_price = model.predict(X_latest)
predicted_stock_price = scaler.inverse_transform(predicted_stock_price)

print("Predicted Gold Prices for the next day: ", predicted_stock_price)

ValueError: cannot reshape array of size 49 into shape (60,)

In [ ]:
# Predict the next 4 days iteratively
predicted_prices = []
current_batch = scaled_data[-60:].reshape(1, 60, 1)  # Most recent 60 days

for i in range(4):  # Predicting 4 days
    # Get the prediction (next day)
    next_prediction = model.predict(current_batch)
    
    # Reshape the prediction to fit the batch dimension
    next_prediction_reshaped = next_prediction.reshape(1, 1, 1)
    
    # Append the prediction to the batch used for predicting
    current_batch = np.append(current_batch[:, 1:, :], next_prediction_reshaped, axis=1)
    
    # Inverse transform the prediction to the original price scale
    predicted_prices.append(scaler.inverse_transform(next_prediction)[0, 0])

print("Predicted Gold Prices for the next 4 days: ", predicted_prices)

In [ ]:
!pip install mplfinance -qqq
import pandas as pd
import mplfinance as mpf
import matplotlib.dates as mpl_dates
import matplotlib.pyplot as plt

# Assuming 'data' is your DataFrame with the fetched AAPL stock data
# Make sure it contains Open, High, Low, Close, and Volume columns

# Creating a list of dates for the predictions
last_date = test_data.index[-1]
next_day = last_date + pd.Timedelta(days=1)
prediction_dates = pd.date_range(start=next_day, periods=4)

# Assuming 'predicted_prices' is your list of predicted prices for the next 4 days
predictions_df = pd.DataFrame(index=prediction_dates, data=predicted_prices, columns=['Close'])

# Plotting the actual data with mplfinance
mpf.plot(test_data, type='candle', style='charles', volume=True)

# Overlaying the predicted data
plt.figure(figsize=(10,6))
plt.plot(predictions_df.index, predictions_df['Close'], linestyle='dashed', marker='o', color='red')

plt.title("AAPL Stock Price with Predicted Next 4 Days")

In [ ]:
import pandas as pd
import mplfinance as mpf
import matplotlib.dates as mpl_dates
import matplotlib.pyplot as plt

# Fetch the latest 60 days of AAPL stock data
data = yf.download('GC=F', period='3mo', interval='1d') # Fetch 64 days to display last 60 days in the chart

# Select 'Close' price and scale it
closing_prices = data['Close'].values.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(closing_prices)

# Predict the next 4 days iteratively
predicted_prices = []
current_batch = scaled_data[-60:].reshape(1, 60, 1)  # Most recent 60 days

for i in range(4):  # Predicting 4 days
    next_prediction = model.predict(current_batch)
    next_prediction_reshaped = next_prediction.reshape(1, 1, 1)
    current_batch = np.append(current_batch[:, 1:, :], next_prediction_reshaped, axis=1)
    predicted_prices.append(scaler.inverse_transform(next_prediction)[0, 0])

# Creating a list of dates for the predictions
last_date = data.index[-1]
next_day = last_date + pd.Timedelta(days=1)
prediction_dates = pd.date_range(start=next_day, periods=4)

# Adding predictions to the DataFrame
predicted_data = pd.DataFrame(index=prediction_dates, data=predicted_prices, columns=['Close'])

# Combining both actual and predicted data
combined_data = pd.concat([data['Close'], predicted_data['Close']])
combined_data = combined_data[-64:] # Last 60 days of actual data + 4 days of predictions

# Plotting the actual data
plt.figure(figsize=(10,6))
plt.plot(data.index[-60:], data['Close'][-60:], linestyle='-', marker='o', color='blue', label='Actual Data')

# Plotting the predicted data
plt.plot(prediction_dates, predicted_prices, linestyle='-', marker='o', color='red', label='Predicted Data')

plt.title("Gold Price: Last 60 Days and Next 4 Days Predicted")
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()